In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import requests
from config import g_key



Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [2]:
#load csv to dataframe
df = pd.read_csv('../../output_data/city_data.csv').drop(columns='Unnamed: 0')
df.head()

,city,lat,lng,temp,humidity,wind,cloudiness,country,date
0,christchurch,-43.5333,172.6333,38.03,100,2.30,2,NZ,Wed Jun 2 15:36:14 2021
1,richards bay,-28.7830,32.0377,61.05,80,6.42,1,ZA,Wed Jun 2 15:25:42 2021
2,bredasdorp,-34.5322,20.0403,55.27,94,8.08,31,ZA,Wed Jun 2 15:36:16 2021
3,hithadhoo,-0.6000,73.0833,83.71,68,16.71,99,MV,Wed Jun 2 15:36:17 2021
4,vaini,-21.2000,-175.2000,69.96,83,2.30,75,TO,Wed Jun 2 15:36:18 2021


 Humidity Heatmap

Configure gmaps.
Use the Lat and Lng as locations and Humidity as the weight.
Add Heatmap layer to map.

In [3]:
#configure
gmaps.configure(api_key=g_key)

#use loc for lat and lng
loc = df[['lat','lng']]
humidity = df.humidity

#create map
fig = gmaps.figure(center=(10.0,-5.0),zoom_level=2)
heat_layer=gmaps.heatmap_layer(loc,weights=humidity,dissipating=False,max_intensity=300,point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

 Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.
Drop any rows will null values.

In [4]:
#df.city.count()
df = df.dropna()
df.city.count()

572

 Hotel Map

Store into variable named hotel_df.

Add a "Hotel Name" column to the DataFrame.

Set parameters to search for hotels with 5000 meters.

Hit the Google Places API for each city's coordinates.

Store the first Hotel result into the DataFrame.

Plot markers on top of the heatmap.

In [5]:
hotel_df = df[(df.temp<80)&(df.temp>70)&(df.wind<10)&(df.cloudiness==0)].copy()
hotel_df

,city,lat,lng,temp,humidity,wind,cloudiness,country,date
168,gat,31.6100,34.7642,72.81,100,7.18,0,IL,Wed Jun 2 15:38:28 2021
205,arcachon,44.5875,-1.0519,77.32,57,8.05,0,FR,Wed Jun 2 15:38:50 2021
438,koungou,-12.7336,45.2042,73.11,88,5.70,0,YT,Wed Jun 2 15:41:18 2021
472,zhanaozen,43.3000,52.8000,78.28,35,5.48,0,KZ,Wed Jun 2 15:42:06 2021


In [6]:
hotel_df['Hotel Name'] = ''
hotel_df.head()

,city,lat,lng,temp,humidity,wind,cloudiness,country,date,Hotel Name
168,gat,31.6100,34.7642,72.81,100,7.18,0,IL,Wed Jun 2 15:38:28 2021,
205,arcachon,44.5875,-1.0519,77.32,57,8.05,0,FR,Wed Jun 2 15:38:50 2021,
438,koungou,-12.7336,45.2042,73.11,88,5.70,0,YT,Wed Jun 2 15:41:18 2021,
472,zhanaozen,43.3000,52.8000,78.28,35,5.48,0,KZ,Wed Jun 2 15:42:06 2021,


In [7]:
#set params
params={
    'radius':5000,
    'types':'lodging',
    'key':g_key
}


for i,row in hotel_df.iterrows():
    
    url='https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    params['location'] = f'{row.lat}, {row.lng}'
    res = requests.get(url,params).json()
    hotel_df.loc[i,'Hotel Name'] = res['results'][0]['name']
hotel_df.rename(columns={'city':'City','lat':'Lat','lng':'Lng','country':'Country'},inplace=True)

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))